# Blob Tips

- Blob の接続文字列から Account Key などを取得
- Blob URL から Blob name の取得

In [ ]:
%pip install python-dotenv

## Blob の接続文字列から Account Key などを取得

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

connection_str = os.getenv('BLOB_CONNECTION_STRING')
print(f"{connection_str=}")

account_name = connection_str.split(';')[1].split('AccountName=')[1]
print(f"{account_name=}")

account_key = connection_str.split(';')[2].split('AccountKey=')[1]
print(f"{account_key=}")

endpoint_suffix = connection_str.split(';')[3].split('EndpointSuffix=')[1]
print(f"{endpoint_suffix=}")

connection_str='DefaultEndpointsProtocol=https;AccountName=styokosandbox;AccountKey=J5BjsRyuAE0LH/O5ZynxsNaDFv0YFnsf6ONpm0eict9DsxPm0KbDvCBhJE2eQp47YZ2UOtPcGlH8GslHY9kZMQ==;EndpointSuffix=core.windows.net'
account_name='styokosandbox'
account_key='J5BjsRyuAE0LH/O5ZynxsNaDFv0YFnsf6ONpm0eict9DsxPm0KbDvCBhJE2eQp47YZ2UOtPcGlH8GslHY9kZMQ=='
endpoint_suffix='core.windows.net'


In [5]:
def get_blob_account_url_from_connection_string(conn: str) -> str:
    splitted_conn = conn.split(';')
    account_name = splitted_conn[1].split('AccountName=')[1]
    endpoint_suffix = splitted_conn[3].split('EndpointSuffix=')[1]
    return f"https://{account_name}.blob.{endpoint_suffix}"

connection_str = os.getenv('BLOB_CONNECTION_STRING')
get_blob_account_url_from_connection_string(connection_str)

'https://styokosandbox.blob.core.windows.net'

## Blob URL から Blob name の取得

blob name に仮想ディレクトリなしの場合はシンプルに取得できるが、"/" が含まれる場合は面倒。

In [9]:
# ユースケースとして EventGrid Trigger で BLOB の URL のみがわかる状態で、blob name を抽出したい場合
BLOB_NAME = "sample.txt" # この値を取得したい
BLOB_CONTAINER_NAME: str = "articles"
BLOB_URL = f"https://styokosandbox.blob.core.windows.net/{BLOB_CONTAINER_NAME}/{BLOB_NAME}"


def get_simple_blob_name(blob_url: str) -> str:
    """
    blob name に絶対に '/' が含まれない場合のみ有効
    """
    return blob_url.split('/')[-1]


def get_blob_name_from_blob_url(blob_url: str, container_name: str) -> str:
    """
    blob name '/' が含まれる場合など全てのケースで有効
    """
    conn = os.getenv('BLOB_CONNECTION_STRING')
    account_url = get_blob_account_url_from_connection_string(conn)
    prefix = f"{account_url}/{BLOB_CONTAINER_NAME}/"
    return blob_url.split(prefix)[1]

simple_process_name = get_simple_blob_name(BLOB_URL)
print(f"{simple_process_name=}")
correct_name = get_blob_name_from_blob_url(BLOB_URL, BLOB_CONTAINER_NAME)
print(f"{correct_name=}")


simple_process_name='sample.txt'
correct_name='test1/nest/sample.txt'
